In [ ]:
from docling.document_converter import DocumentConverter
import json

source = r"/home/ubuntu/Desktop/OmniPDF/sample-files/sample-tables.pdf"  # document per local path or URL

converter = DocumentConverter()
doc = converter.convert(source).document

data = doc.export_to_dict()

with open("./output.json", "w") as f:
    json.dump(data, f, indent=4)

c:\Users\limjj\Documents\OTB\OmniPDF\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import requests

# system_prompt = "Translate the following from english to chinese"


#################### Qwen setup ####################
# LLM_URL = "http://192.168.1.108:8000/v1/chat/completions" #chat/completions  
LLM_URL = "http://192.168.1.197:80/v1/chat/completions"

TOKEN = "token-abc123"

def translate(prompt, input_lang=None, output_lang="English"):
    if input_lang:
        system_prompt = (
            f"You are a professional translator. Given the input language '{input_lang}', "
            f"think deeply and translate the following to '{output_lang}'. "
            f"Return only the translated text."
        )
    else:
        system_prompt = (
            f"You are a professional translator. Think deeply and translate the following to '{output_lang}'. "
            f"Detect the source language automatically and return only the translated text."
        )

    r = requests.post(
        LLM_URL,
        headers={
            "Content-Type": "application/json",
            "Authorization": f"Bearer {TOKEN}"
        },
        json={
            "model": "qwen2.5",
            "messages": [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": prompt}
            ],
            "temperature": 0
        }
    )

    return r.json()["choices"][0]["message"]["content"]

translate("I took the bus to school", input_lang="english", output_lang="malay")

KeyboardInterrupt: 

### Handle texts

In [17]:
input_lang="english"
output_lang="malay"

for entry in data["texts"]:
    original_text = entry.get("text") or entry.get("orig")
    if original_text:
        translated_text = translate(original_text, input_lang=input_lang, output_lang=output_lang)
        entry["trans"] = translated_text

with open("translated.json", "w") as f:
    json.dump(data, f, indent=4)


### Handle table (table cells)

In [19]:
for table in data["tables"]:
    for entry in table["data"]["table_cells"]:
        original_text = entry.get("text")
        if original_text:
            translated_text = translate(original_text, input_lang=input_lang, output_lang=output_lang)
            entry["trans"] = translated_text

In [20]:
with open("translated_more.json", "w") as f:
    json.dump(data, f, indent=4)